# Introduction

A group of city planners are interested in how different neighborhoods in the city of London perform with regards to recycling. They would also like to understand if the presence of recycling centers provide an advantage to the neighborhood in terms of their score. This will enable them to understand whether they should invest in the infrastructure of new centers, or rather in public awareness campaigns to help better the recycling of each community.

Foursquare will provide us with the geolocations of all recycling centers in the different neighbourhoods of London.

This will be combined with the recycling rates given by the city of London to all its' neighborhoods.
https://data.london.gov.uk/dataset/household-waste-recycling-rates-borough

I will superimpose a chloropleth map over the city of London, and colour each square according to it's recycling rate score. Recycing centers will be represented as circular points, information collected from foursquare.
Finally, a table will be produced, putting recycling scores next to recycling center counts, which will enable us to determine if there is any correlation between the better recycling of some communities, and their proximity to more centers.

In [23]:
#Import Libraries to perform Foursquare API query

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [54]:
#Implement Foursquare credentials

CLIENT_ID = '2EX3X52W403513S2YBE43IEV2S5JS3RXGEXVKVPTRHS1GFBV' # your Foursquare ID
CLIENT_SECRET = 'XRDQAXZDAUEMFOMW5JNR5J2JKAXJPYWT0FFEAVWGA0EGSVY2' # your Foursquare Secret
VERSION = '20191231'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2EX3X52W403513S2YBE43IEV2S5JS3RXGEXVKVPTRHS1GFBV
CLIENT_SECRET:XRDQAXZDAUEMFOMW5JNR5J2JKAXJPYWT0FFEAVWGA0EGSVY2


In [55]:
address = 'London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5073219 -0.1276474


In [56]:
search_query = 'Recycling'
radius = 70000
print(search_query + ' .... OK!')

Recycling .... OK!


In [57]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2EX3X52W403513S2YBE43IEV2S5JS3RXGEXVKVPTRHS1GFBV&client_secret=XRDQAXZDAUEMFOMW5JNR5J2JKAXJPYWT0FFEAVWGA0EGSVY2&ll=51.5073219,-0.1276474&v=20191231&query=Recycling&radius=70000&limit=30'

In [58]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f134b0767666079e50f948e'},
 'response': {'venues': [{'id': '5d9c82e8c02e5d00087a88ff',
    'name': 'Londons Computer Recycling',
    'location': {'address': 'Unit B, Pegasus Works 8-10 Roebuck Road, Hainault Business Park',
     'lat': 51.50980284589468,
     'lng': -0.12448668479919432,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.50980284589468,
       'lng': -0.12448668479919432}],
     'distance': 352,
     'postalCode': 'IG6 3UF',
     'cc': 'GB',
     'city': 'Ilford',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['Unit B, Pegasus Works 8-10 Roebuck Road, Hainault Business Park',
      'Ilford',
      'Greater London',
      'IG6 3UF',
      'United Kingdom']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b36',
      'name': 'IT Services',
      'pluralName': 'IT Services',
      'shortName': 'IT Services',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/

In [59]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\Docherty.ATGCH\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,5d9c82e8c02e5d00087a88ff,Londons Computer Recycling,"[{'id': '52f2ab2ebcbc57f1066b8b36', 'name': 'I...",v-1595099656,False,"Unit B, Pegasus Works 8-10 Roebuck Road, Haina...",51.509803,-0.124487,"[{'label': 'display', 'lat': 51.50980284589468...",352,IG6 3UF,GB,Ilford,Greater London,United Kingdom,"[Unit B, Pegasus Works 8-10 Roebuck Road, Hain...",NaN,NaN,NaN
1,4c26135aa852c9284ee4e56c,Recycling,"[{'id': '4bf58dd8d48988d115951735', 'name': 'B...",v-1595099656,False,100 Elephant Road,51.494712,-0.098254,"[{'label': 'display', 'lat': 51.49471156511171...",2473,NaN,GB,Camberwell,Greater London,United Kingdom,"[100 Elephant Road, Camberwell, Greater London...",NaN,NaN,NaN
2,4adb5564f964a520042621e3,Reuse and Recycling Centre,[],v-1595099656,False,Homsey St.,51.549786,-0.110673,"[{'label': 'display', 'lat': 51.54978608661062...",4871,NaN,GB,Islington,Greater London,United Kingdom,"[Homsey St., Islington, Greater London, United...",NaN,NaN,NaN
3,4bd435acb221c9b6f226dbd0,Recycling Centre,"[{'id': '4f4531084b9074f6e4fb0101', 'name': 'R...",v-1595099656,False,Factory Ln,51.377536,-0.115076,"[{'label': 'display', 'lat': 51.37753644977933...",14473,CR0 3RL,GB,Croydon,Greater London,United Kingdom,"[Factory Ln, Croydon, Greater London, CR0 3RL,...",NaN,NaN,NaN
4,4b604c5ef964a5204ade29e3,Reuse and Recycling Centre,"[{'id': '4f4531084b9074f6e4fb0101', 'name': 'R...",v-1595099656,False,Townmead Rd.,51.475169,-0.275310,"[{'label': 'display', 'lat': 51.47516930275923...",10842,TW9 4EL,GB,Kew,Greater London,United Kingdom,"[Townmead Rd. (at Mortlake Rd.), Kew, Greater ...",at Mortlake Rd.,NaN,NaN


In [42]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Londons Computer Recycling,IT Services,"Unit B, Pegasus Works 8-10 Roebuck Road, Haina...",51.509803,-0.124487,"[{'label': 'display', 'lat': 51.50980284589468...",352,IG6 3UF,GB,Ilford,Greater London,United Kingdom,"[Unit B, Pegasus Works 8-10 Roebuck Road, Hain...",NaN,NaN,5d9c82e8c02e5d00087a88ff
1,Recycling,Bike Shop,100 Elephant Road,51.494712,-0.098254,"[{'label': 'display', 'lat': 51.49471156511171...",2473,NaN,GB,Camberwell,Greater London,United Kingdom,"[100 Elephant Road, Camberwell, Greater London...",NaN,NaN,4c26135aa852c9284ee4e56c
2,Recycling Centre,Recycling Facility,Factory Ln,51.377536,-0.115076,"[{'label': 'display', 'lat': 51.37753644977933...",14473,CR0 3RL,GB,Croydon,Greater London,United Kingdom,"[Factory Ln, Croydon, Greater London, CR0 3RL,...",NaN,NaN,4bd435acb221c9b6f226dbd0
3,Reuse and Recycling Centre,None,Homsey St.,51.549786,-0.110673,"[{'label': 'display', 'lat': 51.54978608661062...",4871,NaN,GB,Islington,Greater London,United Kingdom,"[Homsey St., Islington, Greater London, United...",NaN,NaN,4adb5564f964a520042621e3
4,Reuse and Recycling Centre,Recycling Facility,Townmead Rd.,51.475169,-0.275310,"[{'label': 'display', 'lat': 51.47516930275923...",10842,TW9 4EL,GB,Kew,Greater London,United Kingdom,"[Townmead Rd. (at Mortlake Rd.), Kew, Greater ...",at Mortlake Rd.,NaN,4b604c5ef964a5204ade29e3
5,"Waste, Refuse and Recycling Centre",Recycling Facility,Forward Dr,51.589479,-0.326443,"[{'label': 'display', 'lat': 51.58947858114041...",16523,HA3 8NT,GB,Wealdstone,Greater London,United Kingdom,"[Forward Dr, Wealdstone, Greater London, HA3 8...",NaN,NaN,4cc44fc9be40a35dabb17d4c
6,Recycling Centre,Recycling Facility,NaN,51.503410,-0.079262,"[{'label': 'display', 'lat': 51.50341, 'lng': ...",3380,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,519bb6ab498eba8880c49496
7,Re-use and Recycling Centre,Recycling Facility,Kimpton Park Way,51.378968,-0.211923,"[{'label': 'display', 'lat': 51.37896769069795...",15438,SM3 9QH,GB,Sutton,Greater London,United Kingdom,"[Kimpton Park Way, Sutton, Greater London, SM3...",NaN,"Lower Morden, London, Greater London",4cb19a1f39458cfafca70ca0
8,Reuse and Recycling Centre,Recycling Facility,Abbey Rd,51.537874,-0.276332,"[{'label': 'display', 'lat': 51.53787408285695...",10845,NW10 7TJ,GB,London,Greater London,United Kingdom,"[Abbey Rd, Brent, Greater London, NW10 7TJ, Un...",NaN,NaN,4d6134e4b6b9a1cd448e8451
9,Guilford Place Recycling Centre,Government Building,Guilford Place,51.523511,-0.119270,"[{'label': 'display', 'lat': 51.523511, 'lng':...",1893,NaN,GB,London,Greater London,United Kingdom,"[Guilford Place (at Guilford St), Camden, Grea...",at Guilford St,NaN,4d9b6254422ea1cdff5dfb4c


In [60]:
dataframe_filtered.name

0                           Londons Computer Recycling
1                                            Recycling
2                                     Recycling Centre
3                           Reuse and Recycling Centre
4                           Reuse and Recycling Centre
5                   Waste, Refuse and Recycling Centre
6                                     Recycling Centre
7                          Re-use and Recycling Centre
8                           Reuse and Recycling Centre
9                      Guilford Place Recycling Centre
10                                    Recycling Centre
11                      Old Ford Water Recycling Plant
12                                     N J B Recycling
13                          Sanctuary Street Recycling
14                          Reuse and Recycling Centre
15               Waldo Road Reuse and Recycling Centre
16                         Re-use and Recycling Centre
17        Churchfields Road Reuse and Recycling Centre
18        

In [61]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [49]:
#Download data related to recycling

url_rec_csv = 'https://data.london.gov.uk/download/household-waste-recycling-rates-borough/15ddc38a-0a37-4f69-98b5-e69e549b39d3/Household%20recycling.csv'
dfrec = pd.read_csv(url_rec_csv)
dfrec

,Code,Area,Year,Recycling_Rates
0,E09000001,City of London,2003/04,8
1,E09000002,Barking and Dagenham,2003/04,5
2,E09000003,Barnet,2003/04,16
3,E09000004,Bexley,2003/04,21
4,E09000005,Brent,2003/04,10
...,...,...,...,...
683,E12000006,East,2018/19,49
684,E12000007,London,2018/19,33
685,E12000008,South East,2018/19,47
686,E12000009,South West,2018/19,50


In [51]:
dfrec_current = dfrec[(dfrec.Year == '2018/19')]
dfrec_current

,Code,Area,Year,Recycling_Rates
645,E09000001,City of London,2018/19,30
646,E09000002,Barking and Dagenham,2018/19,24
647,E09000003,Barnet,2018/19,35
648,E09000004,Bexley,2018/19,54
649,E09000005,Brent,2018/19,37
650,E09000006,Bromley,2018/19,50
651,E09000007,Camden,2018/19,31
652,E09000008,Croydon,2018/19,47
653,E09000009,Ealing,2018/19,53
654,E09000010,Enfield,2018/19,33


In [53]:
dfrec_current.shape

(43, 4)

In [ ]:
url_chloro_LD = 'https://data.london.gov.uk/download/statistical-gis-boundary-files-london/08d31995-dd27-423c-a987-57fe8e952990/London-wards-2018.zip'

In [ ]:
url_chloro_Borough = 'https://ons-inspire.esriuk.com/arcgis/rest/services/Other_Boundaries/London_Assembly_Constituencies_December_2018_Boundaries_EN_BUC_V2/MapServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
https://services1.arcgis.com/HumUw0sDQHwJuboT/arcgis/rest/services/London_Boroughs/FeatureServer/0

In [65]:
# set the filepath and load in a shapefile
fp = “Documents/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp”
map_df = gpd.read_file(fp)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
map_df.head()

SyntaxError: invalid character in identifier (<ipython-input-65-2f7d5596fa97>, line 2)